In [27]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
import pickle
import lzma

In [28]:
# Load data
file = "listings_with_locations.csv"

file_path = Path("../data/" + file)
listings_df = pd.read_csv(file_path, index_col="mls_id", header=0)


listings_df.head()

,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude
mls_id,,,,,,,,,,,,,
E8018446,https://toronto.listing.ca/286-main-st-911.E80...,286 Main St 911,619900,1,1,1,Main St,East End-Danforth,Toronto,condo_apartment,2024-01-30,43.687469,-79.301861
C7266728,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,529000,1,1,0,Queen St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.650343,-79.387806
W7239426,https://toronto.listing.ca/10-park-lawn-rd-140...,10 Park Lawn Rd 1408,624900,1,1,1,Park Lawn Rd,Mimico,Toronto,condo_apartment,2024-01-30,43.622946,-79.481658
E8030950,https://toronto.listing.ca/665-queen-st-402.E8...,665 Queen St 402,899900,2,2,0,Queen St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.658361,-79.351159
E8030860,https://toronto.listing.ca/1190-dundas-st-925....,1190 Dundas St 925,599900,1,1,0,Dundas St,South Riverdale,Toronto,condo_apartment,2024-01-30,43.665267,-79.341034


In [29]:
# Remove luxury listings with more than 5 bathrooms or more than 4 beds 
listings_df = listings_df[(listings_df['baths'] != 0) & (listings_df['baths'] <= 5) & (listings_df['beds'] <= 4) & (listings_df['beds'] != 0) & (listings_df['dens'] <= 2)  & (listings_df['price'] <= 2000000)]

# Remove rows with missing latitude values
listings_df = listings_df.dropna(subset=["latitude"])

In [30]:
# Function to remove outliers (IQR)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers for each property type

for i in range(4):
    cleaned_df = pd.DataFrame()
    for property_type in listings_df['neighbourhood'].unique():
        subset_df = listings_df[listings_df['neighbourhood'] == property_type]
        cleaned_subset = remove_outliers(subset_df, 'price')
        cleaned_df = pd.concat([cleaned_df, cleaned_subset])
    listings_df = cleaned_df

    cleaned_df = pd.DataFrame()
    for property_type in listings_df['property_type'].unique():
        subset_df = listings_df[listings_df['property_type'] == property_type]
        cleaned_subset = remove_outliers(subset_df, 'price')
        cleaned_df = pd.concat([cleaned_df, cleaned_subset])
    listings_df = cleaned_df

In [31]:
listings_df = listings_df[listings_df.groupby('neighbourhood').city.transform('count')>10].copy() 

In [32]:
listings_df.count()

url              2862
address          2862
price            2862
baths            2862
beds             2862
dens             2862
street           2862
neighbourhood    2862
city             2862
property_type    2862
date_scraped     2862
latitude         2862
longitude        2862
dtype: int64

In [33]:
# Feature engineering
listings_df['rel_latitude'] = listings_df['latitude'] - 43
listings_df['rel_longitude'] = listings_df['longitude'] + 79

listings_df.head()

,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude,rel_latitude,rel_longitude
mls_id,,,,,,,,,,,,,,,
C7266728,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,529000,1,1,0,Queen St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.650343,-79.387806,0.650343,-0.387806
C8030612,https://toronto.listing.ca/35-mercer-st-2810.C...,35 Mercer St 2810,1039000,2,2,0,Mercer St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.645471,-79.391166,0.645471,-0.391166
C8030426,https://toronto.listing.ca/55-mercer-st-710.C8...,55 Mercer St 710,999000,2,1,1,Mercer St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.645397,-79.391362,0.645397,-0.391362
C8030300,https://toronto.listing.ca/35-mercer-st-3711.C...,35 Mercer St 3711,1037000,2,2,0,Mercer St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.645471,-79.391166,0.645471,-0.391166
C8029936,https://toronto.listing.ca/608-richmond-st-100...,608 Richmond St 1001,665000,1,1,1,Richmond St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.646841,-79.402888,0.646841,-0.402888


In [34]:
listings_df.nunique()

url              2862
address          2859
price             825
baths               5
beds                4
dens                3
street            699
neighbourhood      80
city                1
property_type       4
date_scraped        1
latitude         1402
longitude        1403
rel_latitude     1402
rel_longitude    1403
dtype: int64

In [35]:
listings_df['neighbourhood'].value_counts()

neighbourhood
Waterfront Communities C1     327
Church-Yonge Corridor         187
Mimico                        149
Islington-City Centre West    101
Bay Street Corridor           100
                             ... 
Edenbridge-Humber Valley       12
Humewood-Cedarvale             12
Greenwood-Coxwell              12
Yonge-Eglinton                 11
North St. James Town           11
Name: count, Length: 80, dtype: int64

In [36]:
# Prepare data for training
cat_columns = ["neighbourhood", "property_type"]
num_columns = ["baths", "beds", "dens"]
y = listings_df["price"]

In [37]:
listings_df_prepared = pd.get_dummies(listings_df[cat_columns], dtype=int)
listings_df_prepared.head()

,neighbourhood_Agincourt South-Malvern West,neighbourhood_Alderwood,neighbourhood_Annex,neighbourhood_Banbury-Don Mills,neighbourhood_Bay Street Corridor,neighbourhood_Bayview Village,neighbourhood_Bendale,neighbourhood_Birchcliffe-Cliffside,neighbourhood_Black Creek,neighbourhood_Brookhaven-Amesbury,...,neighbourhood_Willowdale East,neighbourhood_Willowdale West,neighbourhood_Woburn,neighbourhood_Yonge-Eglinton,neighbourhood_York University Heights,neighbourhood_Yorkdale-Glen Park,property_type_condo_apartment,property_type_condo_townhome,property_type_detached_home,property_type_freehold _townhome
mls_id,,,,,,,,,,,,,,,,,,,,,
C7266728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
C8030612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
C8030426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
C8030300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
C8029936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [38]:
listings_df_prepared[num_columns] = listings_df[num_columns]
listings_df_prepared.head()

,neighbourhood_Agincourt South-Malvern West,neighbourhood_Alderwood,neighbourhood_Annex,neighbourhood_Banbury-Don Mills,neighbourhood_Bay Street Corridor,neighbourhood_Bayview Village,neighbourhood_Bendale,neighbourhood_Birchcliffe-Cliffside,neighbourhood_Black Creek,neighbourhood_Brookhaven-Amesbury,...,neighbourhood_Yonge-Eglinton,neighbourhood_York University Heights,neighbourhood_Yorkdale-Glen Park,property_type_condo_apartment,property_type_condo_townhome,property_type_detached_home,property_type_freehold _townhome,baths,beds,dens
mls_id,,,,,,,,,,,,,,,,,,,,,
C7266728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
C8030612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,2,0
C8030426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,1,1
C8030300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,2,0
C8029936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1


In [39]:
fit_columns = listings_df_prepared.columns

with open("fit_columns.pkl", 'wb') as file:  
    pickle.dump(fit_columns, file)

In [40]:
X = listings_df_prepared
y = listings_df["price"]

In [41]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [42]:
# Create a pipeline with StandardScaler and RandomForestRegressor
rf_pipeline = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=500, random_state=1))

In [43]:
# Train the model and evaluate it using cross-validation
cv_scores = cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE Scores:", -cv_scores)
print("Mean CV MAE:", -cv_scores.mean())

Cross-Validation MAE Scores: [86230.5768675  87716.87069721 80632.62982658 83249.70606982
 81216.546953  ]
Mean CV MAE: 83809.26608282482


In [44]:
# Fit the model on the training set
rf_pipeline.fit(X_train, y_train)

# Make predictions on the test set
predictions_test = rf_pipeline.predict(X_test)

In [45]:
pkl_model = "housingModel_pkl.xz"  

with lzma.open(pkl_model, "wb") as file:
    pickle.dump(rf_pipeline, file)

#with open(pkl_model, 'wb') as file:  
#    pickle.dump(rf_pipeline, file)

In [46]:
# Evaluate the model on the test set
rf_mae_test = mean_absolute_error(y_test, predictions_test)
print("Random Forest Test MAE:", rf_mae_test)

Random Forest Test MAE: 82226.36666766682


In [47]:
# Make predictions on the training set
predictions_train = rf_pipeline.predict(X_train)

In [48]:
test_data = {"Actual": y_test,
        "Predicted": predictions_test}
test_df = pd.DataFrame(test_data)

test_df["ratio"] = abs (test_df["Actual"] - test_df["Predicted"]) / test_df["Actual"]
test_df["Difference"] = abs(test_df["Actual"] - test_df["Predicted"])


train_data = {"Actual": y_train,
        "Predicted": predictions_train}
train_df = pd.DataFrame(train_data)

train_df["ratio"] = abs (train_df["Actual"] - train_df["Predicted"]) / train_df["Actual"]
train_df["Difference"] = abs(train_df["Actual"] - train_df["Predicted"])

comparison_df = pd.concat([test_df, train_df], axis = 0)

og_comparison_df = pd.concat([comparison_df, listings_df], axis = 1)

og_comparison_df.head()

,Actual,Predicted,ratio,Difference,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude,rel_latitude,rel_longitude
mls_id,,,,,,,,,,,,,,,,,,,
W8018028,625000,6.206798e+05,0.006912,4320.218560,https://toronto.listing.ca/2-fieldway-rd-701.W...,2 Fieldway Rd 701,625000,1,1,1,Fieldway Rd,Islington-City Centre West,Toronto,condo_apartment,2024-01-30,43.643148,-79.527023,0.643148,-0.527023
C7240366,1049000,9.293576e+05,0.114054,119642.361490,https://toronto.listing.ca/15-holmes-ave-504.C...,15 Holmes Ave 504,1049000,2,3,0,Holmes Ave,Willowdale East,Toronto,condo_apartment,2024-01-30,43.777961,-79.414241,0.777961,-0.414241
W7254918,589900,6.095072e+05,0.033238,19607.190424,https://toronto.listing.ca/25-neighbourhood-la...,25 Neighbourhood Lane 811,589900,1,1,1,Neighbourhood Lane,Stonegate-Queensway,Toronto,condo_apartment,2024-01-30,43.638218,-79.489363,0.638218,-0.489363
W7294830,1395000,1.325672e+06,0.049698,69328.460333,https://toronto.listing.ca/121-mulock-ave.W729...,121 Mulock Ave,1395000,4,3,1,Mulock Ave,Junction Area,Toronto,freehold _townhome,2024-01-30,43.670984,-79.465552,0.670984,-0.465552
C7358908,619900,6.969784e+05,0.124340,77078.416438,https://toronto.listing.ca/330-adelaide-st-703...,330 Adelaide St 703,619900,1,1,1,Adelaide St,Moss Park,Toronto,condo_apartment,2024-01-30,43.647552,-79.392448,0.647552,-0.392448


In [49]:
sorted_comparison = og_comparison_df.sort_values(by="ratio", ascending=False)
sorted_comparison.head()

,Actual,Predicted,ratio,Difference,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude,rel_latitude,rel_longitude
mls_id,,,,,,,,,,,,,,,,,,,
E7367870,399900,6.605626e+05,0.651819,260662.565873,https://toronto.listing.ca/2800-warden-ave-804...,2800 Warden Ave 804,399900,2,2,1,Warden Ave,L'Amoreaux,Toronto,condo_apartment,2024-01-30,43.792300,-79.314116,0.792300,-0.314116
C7272710,399000,6.578014e+05,0.648625,258801.350997,https://toronto.listing.ca/14-york-st-409.C727...,14 York St 409,399000,1,1,0,York St,Waterfront Communities C1,Toronto,condo_apartment,2024-01-30,43.642037,-79.381615,0.642037,-0.381615
E7387572,699000,1.147309e+06,0.641358,448309.101333,https://toronto.listing.ca/113-mcintosh-st.E73...,113 Mcintosh St,699000,1,2,0,Mcintosh St,Birchcliffe-Cliffside,Toronto,detached_home,2024-01-30,43.706633,-79.252523,0.706633,-0.252523
C7298936,430000,7.007205e+05,0.629583,270720.541931,https://toronto.listing.ca/1-king-st-701.C7298...,1 King St 701,430000,1,1,0,King St,Bay Street Corridor,Toronto,condo_apartment,2024-01-30,43.649016,-79.378048,0.649016,-0.378048
C8007140,399900,6.410684e+05,0.603072,241168.448568,https://toronto.listing.ca/51-trolley-cres-307...,51 Trolley Cres 307,399900,1,1,0,Trolley Cres,Waterfront Communities C8,Toronto,condo_apartment,2024-01-30,43.655871,-79.354950,0.655871,-0.354950


In [50]:
# Neighbourhood ratios (margin of error)
neigh_comparison = og_comparison_df.groupby("neighbourhood")

neigh_comparison["ratio"].mean().sort_values(ascending=False)

neighbourhood
Black Creek                0.159514
O'Connor-Parkview          0.138368
Little Portugal            0.136589
Birchcliffe-Cliffside      0.135431
Rouge E11                  0.132663
                             ...   
Kensington-Chinatown       0.040550
Bendale                    0.035406
Regent Park                0.034979
York University Heights    0.026100
Yonge-Eglinton             0.022073
Name: ratio, Length: 80, dtype: float64

In [51]:
sorted_comparison.loc[sorted_comparison["neighbourhood"] == "Corso Italia-Davenport"]

,Actual,Predicted,ratio,Difference,url,address,price,baths,beds,dens,street,neighbourhood,city,property_type,date_scraped,latitude,longitude,rel_latitude,rel_longitude
mls_id,,,,,,,,,,,,,,,,,,,


In [52]:
sorted_comparison[["price", "beds", "dens"]].max()

price    1999000
beds           4
dens           2
dtype: int64